In [1]:
import PyPDF2
from PyPDF2 import PdfFileReader as r
import pycountry

# File names
file_names = ['FY' + str(num).zfill(2) + '.pdf' for num in range(6, 17)]

# get text
def get_page_text(file_name, pageno=0):
    if (pageno <= 0):
        raise ValueError("Page number can't neither be negative nor zero")
    
    pageno -= 1
    
    pdf = r(open(file_name, 'rb'))
    
    if (pageno > pdf.numPages): raise ValueError("Page index can't be bigger than number of pages")
    
    return pdf.getPage(pageno).extractText()

# Gets all text from a pdf 
def get_all_text(file_name):
    
    # Opens a file
    pdf = r(open(file_name, 'rb'))
    
    # Read all of the texts
    txt = ''.join(
        text for text in 
        [
            pdf.getPage(num).extractText()
            for num in range(pdf.numPages)
        ]
    )
    
    return txt

In [ ]:
# TODO: Replace country names to international form St. to Saint



txt_list = [txt.split('\n') for txt in get_all_text(file_names[0]).split('%\n')]

# Adding first element
refusal_rates = [{
        "country" : pycountry.countries.get(name=txt_list[0][-2]),
        "rate" : float(txt_list[0][-1])
    }]


# Interface for pycountry
def get_country(c):
    return pycountry.countries.get(name=c)

# Gets country name
def country_name(name):
    name = name.replace(
        'St.', 'Saint').replace(
        '-', ' and ').replace(
        'Burma', 'Myanmar').replace(
        'Cape Verde', 'Cabo Verde').replace(
        'British Virgin Islands', 'Virgin Islands, British'
    ).replace('Federated States of Micronesia', 'Micronesia, Federated States of').replace('Hong Kong S. A. R.', 'Hong Kong').replace(
        'East Timor', 'Timor-Leste'
    ).replace('China - Mainland', 'China')
    
    # Remove parenthesis
    if '(' in name or '-' in name:
        name = name.split(' ')[0]
        
        
    try:
        return get_country(name)
    except:
        name = ''.join(c.name for c in list(pycountry.countries) if name in c.name)
        try:
            return get_country(name)
        except:
            return None
    return None
    
    

# print(get_country_rate(txt_list[0]))

for i, item in enumerate(txt_list):
    # If FISCAL YEAR in an item, get the last element from the list item
#     print("{} - {}".format(country_name(item[0]), i))
    print(item)


In [ ]:
txt_list[47]

In [3]:
import json
import requests


In [73]:
# replace - with space-space
# replace korea with - Democratic People's Republic of Korea
# replace British Virgin Islands with Virgin Islands (British)
# replace Hong Kong gibberish with Hong Kong and aggregate the rates
# Add a placeholder for Jerusalem - Capital of Israel
# Convert laos to Lao
# Convert Macau to Macao
# Convert Macedonia to Republic of Macedonia
# Convert Tanzania to Republic of Tanzania
# Convert North Korea to Democratic People's Republic of Korea
# Convert South Korea to Republic of Korea
# Convert Zaire to Congo-Kinshasa
# Convert British Virgin Islands to Virgin Islands (British) 
# Replace St. with Saint
# Convert Syria to Syrian Arab Republic
# Convert Macedonia to Republic of Macedonia
# Convert Great Britain and Northern Ireland to United Kingdom of Great Britain and Northern Ireland
# Convert Central African to Central African Republic
# Convert Venezuela to Bolivarian Republic of Venezuela
# Convert Serbia and montenegro to Serbia
# Convert Brunei to Nation of Brunei
# Convert Kosovo to Republic of Kosovo
# Convert Wallis and futuna islands to Wallis and Futuna
# Convert Hong Kong S.A.R to Hong Kong
# Convert Vatican City to Holy See
# Convert Iran to Islamic Republic of Iran
# Convert Guinea - bissau to Guinea-Bissau
# Convert Netherlands antilles to Curacao

def return_response(country):
    req_url = 'https://restcountries.eu/rest/v2/name/'+ country +'?fullText=true'
    return json.loads(requests.get(req_url).text)

In [18]:
txt_list2 = [txt.split('\n') for txt in get_all_text(file_names[0]).split('%\n')]


error_names_l2 = []

for i, item in enumerate(txt_list2):
#     print("Doing job {}".format(i + 1))
    if (type(return_response(item[0])) != list):
        error_names_l2.append(item[0])

In [39]:
txt_list_all = [
    
]

names_all = []

for i in range(0, len(file_names)):
    txt_list_all.append([txt.split('\n') for txt in get_all_text(file_names[i]).split('%\n')])

for item in txt_list_all:
    for i, itm in enumerate(item):
        names_all.append(itm[0])
        
names_all = list(name.capitalize() for name in set(names_all))

In [46]:
names_all = names_all[1:]




Dominica


In [47]:
error_names = []
for i, name in enumerate(names_all):
    if (type(return_response(name))!=list):
        error_names.append(name)
#     print(name)
#     print(return_response(name))

In [49]:
# for i in error_names:
#     print(i)
    
len(error_names)

114

In [51]:
for name in error_names: print (name)

United nations laissez-passer
Tanzania
Peoples republic of korea - north korea
Zaire
Page 3 of 6
British virgin islands
Unknown nationality or stateless
St. vincent and the grenadines
2
Unknown nationality or stateless
St. vincent and the grenadines
Syria
Macedonia
Great britain and northern ireland
Central african 
St. kitts and nevis
Venezuela
Page 4 of 6
Serbia and montenegro
St. lucia
Brunei
Kosovo
Wallis and futuna islands
Macedonia
Hong kong s. a. r.
Adjusted refusal rate - b-visas 
Page 1 of 6
Country 
1
Page 2 of 5
Guinea - bissau
Serbia and montenegro
Iran
Hong kong (bno hk passport)
Vatican city
Netherlands antilles
Laos
Vietnam
Page 3 of 5
Page 5 of 6
Congo (brazzaville)
Gambia, the
*serbia and montenegro - bearers of serbia and 
Republic of the 
Serbia and montenegro
Palestinian authority 
Great britain and northern ireland
Serbia and 
*fy2016 data are preliminary
Data must be read in conjunction with the explanatory notes.
China - mainland
Great britain and 
*serbia and mo